In [1]:
import networkx as nx
import freeman as fm
import matplotlib.pyplot as plt
from pprint import pprint
import pymysql
import sys
from collections import defaultdict

from setup import mysql_setup
conn = mysql_setup()

def get_infos(dates):
    with conn.cursor() as cursor:
        try:
            cursor.execute('''
                SELECT
                    t.track_name, pl.playlist_name
                FROM
                    Track t 
                    INNER JOIN Album_Track altr USING (id_track)
                    INNER JOIN Album al USING (id_album)
                    INNER JOIN Playlist_Track pltr USING (id_track)
                    INNER JOIN Playlist pl USING (id_playlist)
                WHERE
                    al.release_date >= %s and al.release_date <= %s
                ''', (dates[0], dates[1]))
            ret = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel dar select em playlist")
            return None
    return ret

def faz_grafo(dic, ligacoes):
    
    g = nx.Graph()
    
    for ent in dic:
        g.add_node(ent)
        
    keys = list(dic.keys())
        
    for i in range(len(dic)-1): #musicas
        for pl1 in dic[keys[i]]: #playlists
            for j in range(len(dic)-1-i): #musicas
                count = 0
                for pl2 in dic[keys[j+1+i]]: #playlists
                    if pl1 == pl2:
                        count += 1
                if count >= ligacoes:
                    g.add_edge(keys[i], keys[j+1+i])
    
    nodes_to_remove = []
    
    for node in g.nodes:
        if len(g.edges(node)) < 1:
            nodes_to_remove.append(node)
            
    for node in nodes_to_remove:
        g.remove_node(node)
    
    return g

In [2]:
infos = get_infos(['2010-01-01', '2019-12-31'])
d = defaultdict(list)
for info in infos:
    d[info[0]].append(info[1])

In [3]:
g = faz_grafo(d, 2)

In [4]:
nx.write_gml(g, "decada10_1.gml")
g = fm.load('decada10_1.gml')

In [12]:
g.set_all_nodes('labpos', 'hover')
g.set_all_nodes("size", 5)
g.move('kamada_kawai')
g.draw()

In [6]:
g.interact()

In [7]:
g2 = faz_grafo(d, 3)

In [8]:
nx.write_gml(g2, "decada10_2.gml")
g2 = fm.load('decada10_2.gml')

In [9]:
g2.set_all_nodes('labpos', 'hover')
g2.move('kamada_kawai')
g2.draw()

In [11]:
g2.interact()